# Desafio Técnico - Cientista de Dados Júnior
## Escritório de Dados - Prefeitura do Rio de Janeiro
#### Makalister Andrade da Silva


In [41]:
!pip install basedosdados sqlalchemy pandas
import basedosdados as bd
import pandas as pd
from sqlalchemy import create_engine
from google.cloud import bigquery


### Todas as consultas foram realizadas no dia 11/02/2024, e portanto seus resultados devem levar em consideração o conteúdo do banco de dados nesta data

### Importando tabelas e salvando-as em dataframes

#### Importando a tabela de Chamados e salvando em dataframe para uso posterior

In [42]:
query_chamados = "SELECT * FROM `datario.administracao_servicos_publicos.chamado_1746` WHERE data_inicio BETWEEN '2023-04-01T00:00:00' AND '2023-04-01T23:59:59'"
df_chamados = bd.read_sql(query_chamados, billing_project_id="emd-datario")

Downloading: 100%|██████████| 73/73 [00:00<00:00, 362.21rows/s]


#### Consultando e armazenando os dados de ``bairro`` em um dataframe:

In [43]:
query_bairros = "SELECT * FROM `datario.dados_mestres.bairro`"
df_bairros = bd.read_sql(query_bairros, billing_project_id="emd-datario")

Downloading: 100%|██████████| 164/164 [00:03<00:00, 42.88rows/s]


#### Importando a tabela de Ocupação Hoteleira em Grandes Eventos no Rio

In [44]:
query_eventos = "SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`"
df_eventos = bd.read_sql(query_eventos, billing_project_id="emd-datario")

Downloading: 100%|██████████| 4/4 [00:00<00:00, 22.47rows/s]


### Questão 1

Quantos chamados foram abertos no dia 01/04/2023?

In [45]:

query_conta_chamados = "SELECT count(id_chamado) AS QTD_Chamados FROM `datario.administracao_servicos_publicos.chamado_1746` WHERE data_inicio BETWEEN '2023-04-01T00:00:00' AND '2023-04-01T23:59:59'"
df_conta_chamados = bd.read_sql(query_conta_chamados, billing_project_id="emd-datario")

Downloading: 100%|██████████| 1/1 [00:00<00:00,  4.88rows/s]


In [46]:
df_conta_chamados

,QTD_Chamados
0,73


``Resposta:`` Foram feitos 73 chamados no dia 01/04/2023.

### Questão 2

Qual o tipo de chamado que teve mais reclamações no dia 01/04/2023?

Listando as colunas do dataframe para identificação visual:

In [47]:
sorted(df_chamados.keys())

['categoria',
 'data_alvo_diagnostico',
 'data_alvo_finalizacao',
 'data_fim',
 'data_inicio',
 'data_particao',
 'data_real_diagnostico',
 'dentro_prazo',
 'geometry',
 'id_bairro',
 'id_chamado',
 'id_logradouro',
 'id_subtipo',
 'id_territorialidade',
 'id_tipo',
 'id_unidade_organizacional',
 'id_unidade_organizacional_mae',
 'justificativa_status',
 'latitude',
 'longitude',
 'nome_unidade_organizacional',
 'numero_logradouro',
 'prazo_tipo',
 'prazo_unidade',
 'reclamacoes',
 'situacao',
 'status',
 'subtipo',
 'tempo_prazo',
 'tipo',
 'tipo_situacao',
 'unidade_organizacional_ouvidoria']

A coluna ``'tipo'`` conterá a informação para responder esta questão.

In [48]:
df_chamados['tipo'].value_counts()

Poluição sonora             24
Estacionamento Irregular     9
Iluminação Pública           8
Remoção Gratuita             5
Limpeza                      5
Postura Municipal            4
Ocupação de área pública     3
Pavimentação                 3
Comlurb - Vetores            2
Fiscalização de obras        2
Manejo Arbóreo               2
Programa Cegonha Carioca     1
Ônibus                       1
Drenagem e Saneamento        1
Feiras                       1
Comércio ambulante           1
Semáforo                     1
Name: tipo, dtype: int64

``Resposta:`` Podemos verificar que o tipo de chamado mais frequente no dia 01/04/2023 é ``Poluição sonora``, contendo 24 chamados

### Questão 3

Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

Contando os bairros (``'id_bairro'``) que mais se repetem entre os chamados:

In [49]:
id_bairros_max_chamados = df_chamados['id_bairro'].value_counts().nlargest(3).index.tolist()
id_bairros_max_chamados

['66', '26', '144']

In [50]:
bairros_max_chamados = df_bairros.loc[df_bairros['id_bairro'].isin(id_bairros_max_chamados)]['nome'].tolist()
bairros_max_chamados

['Leblon', 'Engenho de Dentro', 'Campo Grande']

``Resposta:`` Os bairros que mais possuem chamados  são: ``'Leblon'``, ``'Engenho de Dentro'``, ``'Campo Grande'``, no dia 01/04/2023

### Questão 4

Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [51]:
subprefeitura_max_chamados = df_bairros.loc[df_bairros['id_bairro'].isin([id_bairros_max_chamados[0]])]['subprefeitura'].tolist()
subprefeitura_max_chamados

['Zona Norte']

``Resposta:`` A subprefeitura com maior número de chamados no dia 01/04/2023 foi a da ``'Zona Norte'``

### Questão 5

Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [52]:
chamados_sem_b_sub = df_chamados[df_chamados['id_bairro'].isna()].to_dict()
{'tipo': chamados_sem_b_sub['tipo'][0],'subtipo': chamados_sem_b_sub['subtipo'][0]}

{'tipo': 'Ônibus',
 'subtipo': 'Verificação de ar condicionado inoperante no ônibus'}

``Resposta:`` O chamado não foi associado a um bairro ou subprefeitura pois o registro é sobre  ``'Verificação de ar condicionado inoperante no ônibus'``, logo não fora atribuído a nenhuma localidade.

### Questão 6

Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [53]:
total_chamados_perturbacao_sossego = df_chamados[(df_chamados['subtipo'] == 'Perturbação do sossego') &
                                                 (df_chamados['data_inicio'] >= '2022-01-01') &
                                                 (df_chamados['data_inicio'] <= '2023-12-31')].shape[0]

In [54]:
total_chamados_perturbacao_sossego

24

``Resposta:`` Foram feitos 24 chamados relacionados a ``'Pertubação do sossego'``.

### Questão 7

Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [55]:

# Sua consulta SQL atualizada
# Retorna contagem de eventos agrupaos por eventos
query_tot_eventos = """
SELECT e.evento, COUNT(c.id_chamado) AS total_ocorrencias
FROM `datario.administracao_servicos_publicos.chamado_1746` c
INNER JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.subtipo = 'Perturbação do sossego'
AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio')
GROUP BY e.evento
ORDER BY total_ocorrencias DESC;
"""
query_select_eventos = """
SELECT c.*
FROM `datario.administracao_servicos_publicos.chamado_1746` c
INNER JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.subtipo = 'Perturbação do sossego'
AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio')
;
"""
client = bigquery.Client(project="emd-datario")

# Executa a consulta atualizada e converte os resultados para um DataFrame
df_tot = client.query(query_tot_eventos).to_dataframe()
df_select = client.query(query_select_eventos).to_dataframe()

# Exibe o DataFrame
print(df_tot)
print('\n')
print(df_select)

        evento  total_ocorrencias
0  Rock in Rio                518
1     Carnaval                197
2    Reveillon                 79


    id_chamado         data_inicio            data_fim id_bairro  \
0     18078403 2022-12-31 00:08:42                 NaT        74   
1     18079140 2022-12-31 13:57:43                 NaT        89   
2     18077761 2022-12-30 16:48:53 2022-12-30 20:05:29        63   
3     18079523 2022-12-31 20:52:48 2023-01-08 23:40:57       149   
4     17662396 2022-09-03 11:22:22 2022-09-03 17:13:50        59   
..         ...                 ...                 ...       ...   
789   18327681 2023-02-18 05:23:18 2023-02-21 10:51:39        24   
790   18327522 2023-02-18 01:15:43 2023-02-21 10:30:35        74   
791   18329833 2023-02-19 03:33:38 2023-02-27 08:45:25       128   
792   18331103 2023-02-19 20:57:31 2023-02-28 10:01:00        90   
793   18329577 2023-02-18 22:08:19 2023-02-27 08:51:35       121   

    id_territorialidade id_logradouro  numero

``Resposta:``

1.   Rock in Rio 518
2.   Carnaval 197
3.   Reveillon 79.

### Questão 8

Quantos chamados desse subtipo foram abertos em cada evento??

In [56]:
query_chamados_eventos = """
SELECT e.evento, COUNT(c.id_chamado) AS total_ocorrencias
FROM `datario.administracao_servicos_publicos.chamado_1746` c
INNER JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.subtipo = 'Perturbação do sossego'
AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio')
GROUP BY e.evento
ORDER BY total_ocorrencias DESC;
"""
client = bigquery.Client(project="emd-datario")


df_chamados_eventos = client.query(query_chamados_eventos).to_dataframe()

print(df_chamados_eventos)

        evento  total_ocorrencias
0  Rock in Rio                518
1     Carnaval                197
2    Reveillon                 79


``Resposta:``

1.   Rock in Rio 518 chamados
2.   Carnaval 197 chamados
3.   Reveillon 79 chamados.

### Questão 9

Qual evento teve a maior média diária de chamados abertos desse subtipo?


In [57]:
query_medias_diarias ="""SELECT evento AS Evento, ROUND(AVG(QTD_Chamados), 4) AS Media_Diaria_Chamados
FROM (
    SELECT e.evento, DATE(c.data_inicio) AS data, COUNT(*) AS QTD_Chamados
    FROM `datario.administracao_servicos_publicos.chamado_1746` c
    INNER JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
    ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
    WHERE c.subtipo = 'Perturbação do sossego'
    GROUP BY e.evento, DATE(c.data_inicio)
) AS subquery
GROUP BY evento
ORDER BY Media_Diaria_Chamados DESC
"""

client = bigquery.Client(project="emd-datario")

df_select = client.query(query_medias_diarias).to_dataframe()

max_evento = df_select.loc[df_select['Media_Diaria_Chamados'].idxmax()]

print(max_evento)


Evento                   Rock in Rio
Media_Diaria_Chamados          103.6
Name: 0, dtype: object


``Resposta:`` Rock in Rio com a média diária  de 103.6 chamados

### Questão 10

Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.


In [58]:
query_medias_especificos ="""
SELECT evento AS Evento, ROUND(AVG(QTD_Chamados), 2) AS Media_Diaria_Chamados
FROM (
    SELECT e.evento, DATE(c.data_inicio) AS data, COUNT(*) AS QTD_Chamados
    FROM `datario.administracao_servicos_publicos.chamado_1746` c
    INNER JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
    ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
    WHERE c.subtipo = 'Perturbação do sossego'
    GROUP BY e.evento, DATE(c.data_inicio)
) AS subquery
GROUP BY evento
ORDER BY Media_Diaria_Chamados DESC
"""
query_medias_totais ="""
SELECT ROUND(AVG(QTD_Chamados), 2) AS Media_Diaria_Eventos
FROM (
    SELECT DATE(c.data_inicio) AS data, COUNT(*) AS QTD_Chamados
    FROM `datario.administracao_servicos_publicos.chamado_1746` c
    WHERE c.subtipo = 'Perturbação do sossego'
    AND c.data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
    GROUP BY DATE(c.data_inicio)
) AS subquery
"""

client = bigquery.Client(project="emd-datario")

df_select_especificos = client.query(query_medias_especificos).to_dataframe()
df_select_totais = client.query(query_medias_totais).to_dataframe()

print(df_select_especificos)
print('\n')
print(df_select_totais)

        Evento  Media_Diaria_Chamados
0  Rock in Rio                 103.60
1     Carnaval                  65.67
2    Reveillon                  39.50


   Media_Diaria_Eventos
0                  63.2


``Resposta:``

1.   Rock in Rio 103.6 média diária de chamados
2.   Carnaval 65.67 média diária de chamados
3.   Reveillon 39.50 média diária de chamados.
4.   Entre 01/01/2022 e 31/12/2023 a média diária de chamados foi 63.2.